In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
def read_data(pair):
    db = os.path.join(dropbox_dir,'sqlite', 'ploniex-chart-data', f'{pair}.db')
    data = load_trading_data(db, 'chart_data', from_date='2018-01-01', period='30min')
    data['ohlc4'] = (data['open'] + data['close'] + data['high'] + data['low']) / 4
    return data

In [3]:
all_pairs = pd.read_csv('pairs.tsv', header=None)[0].tolist()
data_dict = {pair: read_data(pair) for pair in all_pairs}

# Missing and duplicate candles

In [53]:
def dates_l(start, end, delta):
    size = int((end - start) / delta) + 1
    deltas = pd.Series([delta]*size)
    dates = deltas.cumsum() + start - delta
    return dates.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))

In [49]:
data['date'].iloc[0]

'2018-01-01 00:00:00'

In [66]:
from datetime import timedelta  

for pair in all_pairs:
    data = data_dict[pair]
    min_date = str_time_to_datetime(data['date'].min())
    max_date = str_time_to_datetime(data['date'].max())
    
    dates = set(dates_l(min_date, max_date, timedelta(minutes=30)))
    data_dates = set(data['date'])
    
    missing = list(dates - data_dates)
    date_counts = data['date'].value_counts()
    duplicates = list(date_counts[date_counts > 1].index)
    
    data_dates = data['date'].tolist()
    is_sorted = all(data_dates[i] <= data_dates[i + 1] for i in range(len(data)-1))
            
    print(f"---- {pair} ----")
    print("missing: ", ','.join(missing))
    print("duplicates: ", ','.join(duplicates))
    print("sorted: ", is_sorted)
    print("")

---- BTC_ARDR ----
missing:  
duplicates:  
sorted:  True

---- BTC_BAT ----
missing:  
duplicates:  
sorted:  True

---- BTC_BCHABC ----
missing:  
duplicates:  
sorted:  True

---- BTC_BCHSV ----
missing:  
duplicates:  
sorted:  True

---- BTC_BCN ----
missing:  
duplicates:  
sorted:  True

---- BTC_BNT ----
missing:  2018-12-31 22:00:00,2018-12-31 22:30:00
duplicates:  
sorted:  True

---- BTC_BTS ----
missing:  
duplicates:  
sorted:  True

---- BTC_BURST ----
missing:  
duplicates:  
sorted:  True

---- BTC_CLAM ----
missing:  
duplicates:  
sorted:  True

---- BTC_CVC ----
missing:  
duplicates:  
sorted:  True

---- BTC_DASH ----
missing:  
duplicates:  
sorted:  True

---- BTC_DCR ----
missing:  
duplicates:  
sorted:  True

---- BTC_DGB ----
missing:  
duplicates:  
sorted:  True

---- BTC_DOGE ----
missing:  
duplicates:  
sorted:  True

---- BTC_EOS ----
missing:  
duplicates:  
sorted:  True

---- BTC_ETC ----
missing:  
duplicates:  
sorted:  True

---- BTC_ETH ----
miss

# Pairs Volume

In [67]:
volume_stats = pd.DataFrame()
for pair in all_pairs:
    data = data_dict[pair]
    pair_stats = data[['volume']].describe().transpose()
    pair_stats['sum'] = data['volume'].sum()
    pair_stats['pair'] = pair
    volume_stats = volume_stats.append(pair_stats)

cols = list(volume_stats.columns)
cols = cols[-1:] + cols[:-1]
volume_stats = reset_index_hard(volume_stats).sort_values('mean', ascending=False)[cols]

In [70]:
volume_stats

,pair,count,mean,std,min,25%,50%,75%,max,sum
56,BTC_XRP,"22,188.0000",30.0246,102.5311,0.0000,2.4477,6.9252,19.9406,"3,668.1360","666,186.6484"
16,BTC_ETH,"22,157.0000",28.9813,67.2244,0.0000,3.2500,9.1392,24.9775,"1,529.7998","642,137.5967"
47,BTC_STR,"22,188.0000",13.6252,49.3426,0.0000,0.9732,2.9846,8.9085,"1,617.8589","302,316.8424"
28,BTC_LTC,"22,157.0000",8.6709,20.5478,0.0000,0.8606,2.5083,7.3754,469.6457,"192,121.2045"
3,BTC_BCHSV,"7,227.0000",8.4993,28.6699,0.0000,0.0467,0.5358,4.7490,549.3267,"61,424.5421"
2,BTC_BCHABC,"7,227.0000",7.9730,28.7091,0.0000,0.2046,1.1844,5.0620,"1,053.4070","57,621.0592"
54,BTC_XMR,"22,188.0000",7.7642,14.8834,0.0000,1.5968,3.5871,7.8880,381.5766,"172,271.7686"
15,BTC_ETC,"22,157.0000",4.7180,13.7632,0.0000,0.1227,0.8127,3.2973,379.7992,"104,536.4648"
13,BTC_DOGE,"22,157.0000",4.4980,14.6134,0.0000,0.1573,0.7258,3.1096,635.4568,"99,661.8145"
53,BTC_XEM,"22,188.0000",3.9464,16.5906,0.0000,0.0692,0.4580,2.3849,702.9938,"87,563.5271"


# Find best params

In [6]:
cols = ['window_size', 'target', 'stop_loss', '_return']
aggregated = results.groupby(cols)\
    .mean()\
    .reset_index()

### Based on total profit

In [15]:
DAY = 48
aggregated.query('numberOfTrades > 200')\
    .query(f'maxTimeToClose < {3*DAY}')\
    .sort_values(['totalProfit', 'winPercentage'], ascending=False)\
    .head(10)

,window_size,target,stop_loss,_return,startAmount,numberOfTrades,totalProfit,avgProfit,winPercentage,avgWinTrade,avgLossTrade,profitFactor,maxDrawdown,avgTimeToClose,minTimeToClose,maxTimeToClose
19,4,0.0100,0.0200,-0.0100,100.0000,381.8000,"3,299.4416",6.7844,70.4392,17.0991,-26.1938,1.6956,27.4052,7.6971,0.0000,127.7333
1019,6,0.0100,0.0200,-0.0100,100.0000,269.9667,"1,237.7763",3.8855,74.3092,8.6001,-12.3003,2.1254,21.3913,8.0292,0.0000,111.1000
28,4,0.0100,0.0300,-0.0200,100.0000,202.3667,"1,055.7662",4.3452,81.4935,8.9267,-18.5374,2.3877,18.8168,7.5705,0.0000,116.5333
2019,8,0.0100,0.0200,-0.0100,100.0000,206.8667,720.7784,3.0565,76.5665,6.2855,-9.1463,2.4808,16.3582,8.3047,0.0000,102.5000
118,4,0.0200,0.0200,-0.0200,100.0000,202.9667,554.4705,2.4836,58.7848,9.1042,-8.2767,1.6129,29.0118,11.1069,0.0000,132.0667
18,4,0.0100,0.0200,-0.0200,100.0000,203.1667,510.1014,2.2264,72.4756,5.6903,-8.1827,2.0247,19.8050,5.4399,0.0000,80.4333
9,4,0.0100,0.0100,-0.0100,100.0000,387.5667,230.4218,0.5020,48.4667,3.0470,-2.4567,1.1356,37.5035,3.8630,0.0000,62.7000
109,4,0.0200,0.0100,-0.0100,100.0000,386.0333,212.0654,0.4720,35.5998,4.4572,-2.2680,1.0595,48.3179,7.2180,0.0000,109.1333
1109,6,0.0200,0.0100,-0.0100,100.0000,272.5667,104.4734,0.3685,38.6332,3.9394,-2.2332,1.1448,39.3569,8.1516,0.0000,112.1667
1009,6,0.0100,0.0100,-0.0100,100.0000,273.0000,100.5631,0.3394,51.1828,2.3036,-2.0266,1.2100,31.7454,4.2902,0.0000,59.1000


### Based on win percentage

In [28]:
DAY = 48
selected = aggregated.query('numberOfTrades > 200')\
    .query(f'maxTimeToClose < {3*DAY}')\
    .query('winPercentage > 80')\
    .sort_values(['totalProfit'], ascending=False)\
    .head(10)

selected_first = selected.iloc[0, :]
selected

,window_size,target,stop_loss,_return,startAmount,numberOfTrades,totalProfit,avgProfit,winPercentage,avgWinTrade,avgLossTrade,profitFactor,maxDrawdown,avgTimeToClose,minTimeToClose,maxTimeToClose
28,4,0.0100,0.0300,-0.0200,100.0000,202.3667,"1,055.7662",4.3452,81.4935,8.9267,-18.5374,2.3877,18.8168,7.5705,0.0000,116.5333


In [30]:
cols = ['window_size', 'target', 'stop_loss', '_return']
selected_results = results
for c in cols:
    selected_results = selected_results.query(f'{c} == {selected_first[c]}')

selected_results

,pair,window_size,target,stop_loss,_return,startAmount,numberOfTrades,totalProfit,avgProfit,winPercentage,avgWinTrade,avgLossTrade,profitFactor,maxDrawdown,avgTimeToClose,minTimeToClose,maxTimeToClose
23,BTC_BAT,4,0.0100,0.0300,-0.0200,100.0000,100.0000,128.3782,1.2838,82.0000,2.6535,-4.9561,2.4390,9.8088,10.8400,0.0000,105.0000
7023,BTC_BCHABC,4,0.0100,0.0300,-0.0200,100.0000,70.0000,80.0190,1.1431,77.1429,2.6988,-4.1074,2.2176,11.3924,7.1571,0.0000,53.0000
14026,BTC_BCHSV,4,0.0100,0.0300,-0.0200,100.0000,83.0000,288.3819,3.4745,80.7229,6.6463,-9.8074,2.8378,11.7106,7.3614,0.0000,120.0000
21020,BTC_BCN,4,0.0100,0.0300,-0.0200,100.0000,299.0000,80.9113,0.2706,62.5418,8.7203,-13.8374,1.0522,71.8894,1.0401,0.0000,48.0000
28020,BTC_BTS,4,0.0100,0.0300,-0.0200,100.0000,238.0000,656.4950,2.7584,85.7143,5.9907,-16.6353,2.1607,32.9203,6.0462,0.0000,167.0000
35025,BTC_BURST,4,0.0100,0.0300,-0.0200,100.0000,231.0000,586.7013,2.5398,81.3853,7.2563,-18.0808,1.7546,16.5658,4.6061,0.0000,62.0000
42020,BTC_DASH,4,0.0100,0.0300,-0.0200,100.0000,168.0000,174.9754,1.0415,82.7381,2.7271,-7.0374,1.8574,15.4229,10.8095,0.0000,177.0000
49025,BTC_DCR,4,0.0100,0.0300,-0.0200,100.0000,184.0000,"1,317.3560",7.1595,85.8696,14.1515,-35.3302,2.4341,16.1556,8.7446,0.0000,76.0000
56021,BTC_DOGE,4,0.0100,0.0300,-0.0200,100.0000,207.0000,242.0743,1.1694,75.8454,4.4277,-9.0616,1.5343,22.2794,3.6957,0.0000,85.0000
63027,BTC_ETC,4,0.0100,0.0300,-0.0200,100.0000,169.0000,340.7149,2.0161,84.6154,3.6386,-6.9078,2.8970,8.4941,8.7456,0.0000,115.0000
